In [1]:
import tensorflow as tf

import sys

In [2]:
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image as Img

# For Grad-CAM
import xgradcam
from xgradcam import get_img_array
from xgradcam import make_gradcam_heatmap
from xgradcam import save_and_display_gradcam

from pathlib import Path
import os
import glob
import matplotlib.cm as cm 
from numpy import save
from numpy import load
from skimage.transform import resize
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocessing
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as inceptionresnet_preprocessing
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocessing
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocessing
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_preprocessing
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenetv2_preprocessing
from tensorflow.keras.applications.nasnet import preprocess_input as nasnet_preprocessing
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnetv2_preprocessing
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocessing
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator, DirectoryIterator

In [3]:
def save_and_display_gradcam(img_array, heatmap, cam_path='', alpha=0.4):
    # Load the original image
    img = img_array[0]
    # Rescale heatmap to a range 0-255    
    heatmap = np.uint8(255 * heatmap)
    
    # Resize heatmap to save 
    heatmap_resized = resize(heatmap, (300, 400))
    
    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    return superimposed_img, heatmap_resized

In [ ]:
root_path = "/home/ubuntu/hot-store/final"
model_names = [
    "efficientnet", 
    "inception", 
    "inceptionresnet", 
    "mobilenetv1", 
    "mobilenetv2",
    "nasnetmobile",
    "resnet",
    "resnetv2",
    'vgg',
    "xception"
]
model_dirs = [Path(root_path) / model_name for model_name in model_names]
results_paths = [f"{model_dir}/visualisation/gradcam/" for model_dir in model_dirs]

model_infos = [
    {
        'model_name': 'resnet',
        'model_dir': f'{root_path}/resnet',
        'results_path': f"{root_path}/resnet/visualisation/gradcam/",
        'last_conv_layer_name': 'conv5_block3_out',
        'target_size': (400, 300),
        'preprocessing_function': resnet_preprocessing,    
    },
    {
        'model_name': 'efficientnet',
        'model_dir': f'{root_path}/efficientnet',
        'results_path': f"{root_path}/efficientnet/visualisation/gradcam/",
        'last_conv_layer_name': 'top_conv',
        'target_size': (400, 300),
        'preprocessing_function': efficientnet_preprocessing,    
    },
    {
        'model_name': 'inceptionresnet',
        'model_dir': f'{root_path}/inceptionresnet',
        'results_path': f"{root_path}/inceptionresnet/visualisation/gradcam/",
        'last_conv_layer_name': 'conv_7b_ac',
        'target_size': (400, 300),
        'preprocessing_function': inceptionresnet_preprocessing,    
    },
    {
        'model_name': 'inception',
        'model_dir': f'{root_path}/inception',
        'results_path': f"{root_path}/inception/visualisation/gradcam/",
        'last_conv_layer_name': 'mixed10',
        'target_size': (400, 300),
        'preprocessing_function': inception_preprocessing,    
    },
    {
        'model_name': 'mobilenetv1',
        'model_dir': f'{root_path}/mobilenetv1',
        'results_path': f"{root_path}/mobilenetv1/visualisation/gradcam/",
        'last_conv_layer_name': 'conv_pw_13_relu',
        'target_size': (400, 300),
        'preprocessing_function': mobilenet_preprocessing,    
    },
    {
        'model_name': 'mobilenetv2',
        'model_dir': f'{root_path}/mobilenetv2',
        'results_path': f"{root_path}/mobilenetv2/visualisation/gradcam/",
        'last_conv_layer_name': 'out_relu',
        'target_size': (400, 300),
        'preprocessing_function': mobilenetv2_preprocessing,    
    },
    {
        'model_name': 'nasnetmobile',
        'model_dir': f'{root_path}/nasnetmobile',
        'results_path': f"{root_path}/nasnetmobile/visualisation/gradcam/",
        'last_conv_layer_name': 'activation_484',
        'target_size': (224, 224),
        'preprocessing_function': nasnet_preprocessing,    
    },
    {
        'model_name': 'resnetv2',
        'model_dir': f'{root_path}/resnetv2',
        'results_path': f"{root_path}/resnetv2/visualisation/gradcam/",
        'last_conv_layer_name': 'post_relu',
        'target_size': (400, 300),
        'preprocessing_function': resnetv2_preprocessing,    
    },
    {
        'model_name': 'vgg',
        'model_dir': f'{root_path}/vgg',
        'results_path': f"{root_path}/vgg/visualisation/gradcam/",
        'last_conv_layer_name': 'block5_pool',
        'target_size': (400, 300),
        'preprocessing_function': vgg_preprocessing,    
    },
    {
        'model_name': 'xception',
        'model_dir': f'{root_path}/xception',
        'results_path': f"{root_path}/xception/visualisation/gradcam/",
        'last_conv_layer_name': 'block14_sepconv2_act',
        'target_size': (400, 300),
        'preprocessing_function': xception_preprocessing,    
    },
]

for model_info in model_infos:
    model_weights = glob.glob(f"{model_info['model_dir']}/*.h5")
    
    for model_path in model_weights:
        model_name = Path(model_path).stem
        model_results_path = os.path.join(model_info['results_path'], model_name) 
        # create dir for each model
        if not os.path.exists(model_results_path):
            os.makedirs(model_results_path)

        model = load_model(Path(model_path))
#         model.layers[-1].activation = None
        
        data_generator = ImageDataGenerator(
            fill_mode='nearest',
            preprocessing_function=model_info['preprocessing_function']
        )
        data_iterator = data_generator.flow_from_directory(
            '/home/ubuntu/hot-store/dermx/images/valid', 
            batch_size=1, 
            target_size=model_info['target_size'],
            class_mode='categorical',
            follow_links=True,
            interpolation='bilinear',
            shuffle=False,
        )
        class_indices = data_iterator.class_indices

        for (img_array, label), img_path in zip(data_iterator, data_iterator.filepaths):
            img_name = Path(img_path).stem
            preds = model.predict(img_array)
            for category in class_indices:
                print(model_name, img_name, category)
                heatmap = make_gradcam_heatmap(img_array, model, model_info['last_conv_layer_name'], class_indices[category])
                imposed,heat_map_resized = save_and_display_gradcam(img_array, heatmap)
                # save files
                save(model_results_path + '/' + img_name + '_' +  category + '.npy', heat_map_resized)
                imposed.save(model_results_path + '/' + img_name + '_' +  category + '.png')


Found 524 images belonging to 6 classes.
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 032439HB Acne
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 032439HB Actinic keratosis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 032439HB Psoriasis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 032439HB Seborrheic dermatitis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 032439HB Viral warts
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 032439HB Vitiligo
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 032462HB Acne
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 032462HB Actinic keratosis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 032462HB Psoriasis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 032462HB Seborrheic dermatitis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 032462HB Viral warts
resnet50_r20_

resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 033787HB Acne
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 033787HB Actinic keratosis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 033787HB Psoriasis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 033787HB Seborrheic dermatitis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 033787HB Viral warts
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 033787HB Vitiligo
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 033877HB Acne
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 033877HB Actinic keratosis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 033877HB Psoriasis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 033877HB Seborrheic dermatitis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 033877HB Viral warts
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_blo

resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 034724HB Acne
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 034724HB Actinic keratosis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 034724HB Psoriasis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 034724HB Seborrheic dermatitis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 034724HB Viral warts
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 034724HB Vitiligo
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 034787HB Acne
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 034787HB Actinic keratosis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 034787HB Psoriasis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 034787HB Seborrheic dermatitis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 034787HB Viral warts
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_blo

resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 146--WatermarkedWyJXYXRlcm1hcmtlZCJd Psoriasis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 146--WatermarkedWyJXYXRlcm1hcmtlZCJd Seborrheic dermatitis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 146--WatermarkedWyJXYXRlcm1hcmtlZCJd Viral warts
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 146--WatermarkedWyJXYXRlcm1hcmtlZCJd Vitiligo
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 152--WatermarkedWyJXYXRlcm1hcmtlZCJd Acne
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 152--WatermarkedWyJXYXRlcm1hcmtlZCJd Actinic keratosis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 152--WatermarkedWyJXYXRlcm1hcmtlZCJd Psoriasis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 152--WatermarkedWyJXYXRlcm1hcmtlZCJd Seborrheic dermatitis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 152--WatermarkedWyJX

resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 acne-face-1-37--WatermarkedWyJXYXRlcm1hcmtlZCJd Acne
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 acne-face-1-37--WatermarkedWyJXYXRlcm1hcmtlZCJd Actinic keratosis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 acne-face-1-37--WatermarkedWyJXYXRlcm1hcmtlZCJd Psoriasis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 acne-face-1-37--WatermarkedWyJXYXRlcm1hcmtlZCJd Seborrheic dermatitis
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 acne-face-1-37--WatermarkedWyJXYXRlcm1hcmtlZCJd Viral warts
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 acne-face-1-37--WatermarkedWyJXYXRlcm1hcmtlZCJd Vitiligo
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 acne-face-1-38--WatermarkedWyJXYXRlcm1hcmtlZCJd Acne
resnet50_r20_s0.5_z0.5_b[0.5, 1.5]_lr0.0001_lconv5_block3_out_3 acne-face-1-38--WatermarkedWyJXYXRlcm1hcmtlZCJd Actinic keratosis
resne

...

efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-2-20--WatermarkedWyJXYXRlcm1hcmtlZCJd Vitiligo
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-2-31--WatermarkedWyJXYXRlcm1hcmtlZCJd Acne
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-2-31--WatermarkedWyJXYXRlcm1hcmtlZCJd Actinic keratosis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-2-31--WatermarkedWyJXYXRlcm1hcmtlZCJd Psoriasis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-2-31--WatermarkedWyJXYXRlcm1hcmtlZCJd Seborrheic dermatitis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-2-31--WatermarkedWyJXYXRlcm1hcmtlZCJd Viral warts
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-2-31--WatermarkedWyJXYXRlcm1hcmtlZCJd Vitiligo
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-2-6--WatermarkedWyJXYXRlcm1hcmtlZCJd Acne

efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-4-3--WatermarkedWyJXYXRlcm1hcmtlZCJd Vitiligo
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-4-4--WatermarkedWyJXYXRlcm1hcmtlZCJd Acne
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-4-4--WatermarkedWyJXYXRlcm1hcmtlZCJd Actinic keratosis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-4-4--WatermarkedWyJXYXRlcm1hcmtlZCJd Psoriasis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-4-4--WatermarkedWyJXYXRlcm1hcmtlZCJd Seborrheic dermatitis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-4-4--WatermarkedWyJXYXRlcm1hcmtlZCJd Viral warts
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-4-4--WatermarkedWyJXYXRlcm1hcmtlZCJd Vitiligo
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-56--WatermarkedWyJXYXRlcm1hcmtlZCJd Acne
efficie

efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-face-9--WatermarkedWyJXYXRlcm1hcmtlZCJd Vitiligo
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-scar9--WatermarkedWyJXYXRlcm1hcmtlZCJd Acne
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-scar9--WatermarkedWyJXYXRlcm1hcmtlZCJd Actinic keratosis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-scar9--WatermarkedWyJXYXRlcm1hcmtlZCJd Psoriasis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-scar9--WatermarkedWyJXYXRlcm1hcmtlZCJd Seborrheic dermatitis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-scar9--WatermarkedWyJXYXRlcm1hcmtlZCJd Viral warts
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne-scar9--WatermarkedWyJXYXRlcm1hcmtlZCJd Vitiligo
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 acne1--WatermarkedWyJXYXRlcm1hcmtlZCJd Acne
efficientnetb0_r20_s0.25_z0.5_b[0.

efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016198HB Viral warts
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016198HB Vitiligo
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016207HB Acne
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016207HB Actinic keratosis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016207HB Psoriasis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016207HB Seborrheic dermatitis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016207HB Viral warts
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016207HB Vitiligo
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016263HB Acne
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016263HB Actinic keratosis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016263HB Psoriasis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0

efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016878HB Psoriasis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016878HB Seborrheic dermatitis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016878HB Viral warts
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016878HB Vitiligo
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016933HB Acne
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016933HB Actinic keratosis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016933HB Psoriasis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016933HB Seborrheic dermatitis
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016933HB Viral warts
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016933HB Vitiligo
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]_lr0.0001_lblock6d_add_3 016968HB Acne
efficientnetb0_r20_s0.25_z0.5_b[0.5, 1.5]